<center><img src="https://i.imgur.com/hBPv3fh.png" width="650"></center>
<h1><center>Kaggle LANL Earthquake Prediction Modeling</center></h1>

**Nhiệm vụ**: Dự đoán thời gian sắp xảy ra động đất dựa vào dãy acoustic_data thu được trong mỗi file dữ liệu.

**Model**: Project này sử dụng model CatBoostRegressor. Output của project là một continuous number -> Regression data.

**CatBoost**: Là model được phát triển bởi Yandex dựa trên công thức Gradient Boosting thuộc họ Decision Tree. CatBoost có các ưu điểm:
* Thể hiện tốt mà it cần chỉnh parameters.
* Hỗ trợ các biến categorical không phải dạng số --> Không cần dummies.
* Scale với GPU.
* Độ chính xác cao, có thể giảm overfitting.
* Tốc độ dự đoán nhanh.

[Nguồn](https://catboost.ai/)

In [ ]:
!pip install catboost

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Model
from catboost import CatBoostRegressor, Pool

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [ ]:
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16,
                                                 'time_to_failure': np.float64})
train.head()

Để ý thấy cột **time_to_failure** chứa các con số giống nhau, tuy nhiên nhiều khả năng do số được làm tròn vì khác biệt nằm ở phần thập phân. Cần để pandas hiển thị phần thập phân nhiều hơn để xác nhận điều này.

In [ ]:
pd.options.display.precision = 15 # Hiển thị 15 số thập phân
train.head()

Mỗi file test chứa 150,000 dòng acoustic_data, nhiệm vụ của chúng ta là dự đoán time_to_failure tại dòng cuối cùng của mỗi file test.

<center><b>seg_004cd2</b></center>

|      |acoustic_data|time_to_failure|
|------|-------------|---------------|
|  0   |      6      |               |
|  1   |     16      |               |
| ...  |     ...     |      ...      |
|149998|151          |               |
|149999|77           |    **3.26**   |

<br>

<center><b>submission</b></center>

|          |time_to_failure|
|----------|---------------|
|seg_004cd2|    **3.26**   |
|    ...   |      ...      |
|seg_123ef7|      ...      |

In [ ]:
train_chunk = pd.read_csv('../input/train.csv', chunksize=150_000, iterator=True, dtype={'acoustic_data': np.int16,
                                                                                          'time_to_failure': np.float64})

Do feature gốc chỉ có **acoustic_data**, ta cần sử dụng tới kỹ thuật Feature Engineering để tạo thêm các feature khác dựa trên feature gốc, từ đó tăng khả năng predict của model. Nói cách khác, ta có thêm nhiều dữ kiện hơn để thực hiện việc phỏng đoán thời gian sắp xảy ra động đất.

**Ý tưởng:** Tạo ra function `feature_engineering`, function sẽ tạo ra một list gồm mean, std, min, max của cột acoustic_data (new_feat = \[7.32, 0.16, -77, 109\]). Sau đó biến list này thành pd.Series để append vào 1 pd.DataFrame, mỗi Series append vào DataFrame sẽ trở thành 1 dòng.

In [ ]:
def feature_engineering(df):
    new_feat = []
    new_feat.append(df.mean())
    new_feat.append(df.std())
    new_feat.append(df.min())
    new_feat.append(df.max())
    
    return pd.Series(new_feat)

In [ ]:
# Feature Engineering
X_train = pd.DataFrame()
y_train = pd.Series()

for df in train_chunk:
    fe = feature_engineering(df['acoustic_data'])
    X_train = X_train.append(fe, ignore_index=True)
    y_train = y_train.append(pd.Series(df['time_to_failure'].values[-1])) # Lấy giá trị cuối cùng của mỗi chunk

In [ ]:
# Đặt tên cho các cột
columns = ['mean', 'std', 'min', 'max']
X_train.columns = columns

X_train.head()

In [ ]:
y_train.head()

Các feature được sử dụng là các số liên tiếp và có sự khác biệt lớn về giá trị của chúng. Ví dụ như giá trị của feature **mean** có thể trải từ -10 đến 10, trong khi đó giá trị của feature **max** có thể trải từ 50 đến 200. Do đó, để có thể sử dụng các feature này một cách hiệu quả, chúng ta cần sử dụng kỹ thuật scale để các giá trị của các feature chỉ nằm trong khoảng 0 đến 1 hoặc -1 đến 1 (tùy theo kỹ thuật scale).

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

Tiếp theo, chọn model để train bằng feature đã scale. Ở đây sử dụng model CatBoostRegressor.

In [ ]:
params = {'iterations': 10000,
          'loss_function': 'MAE'}

train_pool = Pool(X_train_scaled, y_train)

cbr = CatBoostRegressor(**params)
cbr.fit(X_train_scaled, y_train, eval_set=train_pool, silent=True)

y_pred = cbr.predict(X_train_scaled)
print('Best score: ' + str((cbr.best_score_)))
print('MAE: {:.3f}'.format(mean_absolute_error(y_train.values, y_pred)))

# SUBMIT

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id') # Đọc file sample_submission để lấy form

X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index) # Tạo DF mới cho X_test để thực hiện Feature Engineering tương tự với train data

In [ ]:
X_test.head()

In [ ]:
# Feature Engineering cho test data
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'mean'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test.head()

In [ ]:
X_test_scaled = scaler.transform(X_test) # Scale feature ở test data

submission['time_to_failure'] = cbr.predict(X_test_scaled)
submission.to_csv('submission.csv')

In [ ]:
submission.head()